In [1]:
from gurobipy import GRB, Model

# Sudoku - Integer Formulation

## Data

In [7]:
init_nums = {
    (1, 1): 8,
    (2, 3): 3, (2, 4): 6,
    (3, 2): 7, (3, 5): 9, (3, 7): 2,
    (4, 2): 5, (4, 6): 7,
    (5, 5): 4, (5, 6): 5, (5, 7): 7,
    (6, 4): 1, (6, 8): 3,
    (7, 3): 1, (7, 8): 6, (7, 9): 8,
    (8, 3): 8, (8, 4): 5, (8, 8): 1,
    (9, 2): 9, (9, 7): 4
}

In [11]:
rows = range(1, 10)
columns = range(1, 10)
boxes = [
    [(3*i + k, 3*j + l) for k in range(1, 4) for l in range(1, 4)]
    for i in range(3) for j in range(3)
]

In [28]:
M = 9

## Model

In [48]:
m = Model("Sudoku integer")

In [49]:
y = m.addVars(rows, columns, vtype=GRB.INTEGER, lb=1, ub=9)

In [50]:
b = m.addVars(rows, columns, rows, columns, vtype=GRB.BINARY)

In [51]:
m.addConstrs(
    y[r, c] == v for ((r, c), v) in init_nums.items()
)
m.update()

unique value per row

In [52]:
for r in rows:
    for c in columns:
        for c1 in columns:
            if c != c1:
                m.addConstr(y[r, c] + 1 <= y[r, c1] + b[r, c, r, c1] * M)
                m.addConstr(y[r, c] >= y[r, c1] + 1 - (1 - b[r, c, r, c1]) * M)

unique value per column

In [53]:
for c in columns:
    for r in rows:
        for r1 in rows:
            if r != r1:
                m.addConstr(y[r, c] + 1 <= y[r1, c] + b[r, c, r1, c] * M)
                m.addConstr(y[r, c] >= y[r1, c] + 1 - (1-b[r, c, r1, c]) * M)

unique value per box

In [54]:
for box in boxes:
    for (r, c) in box:
        for (r1, c1) in box:
            if (r,c) != (r1, c1):
                m.addConstr(y[r, c] + 1 <= y[r1, c1] + b[r, c, r1, c1] * M)
                m.addConstr(y[r, c] >= y[r1, c1] + 1 - (1-b[r, c, r1, c1]) * M)

In [55]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3909 rows, 6642 columns and 11685 nonzeros
Model fingerprint: 0x1a5c6ec6
Variable types: 0 continuous, 6642 integer (6561 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 2298 rows and 5797 columns
Presolve time: 0.05s
Presolved: 1611 rows, 845 columns, 4631 nonzeros
Variable types: 0 continuous, 845 integer (786 binary)

Root relaxation: objective 0.000000e+00, 612 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   19          -    0.00000      -     -    0s
     0     0    0.00000    0   70          -    0.00000      -     -    0s
     0     0    0.00000    0   70          -    0.00000      -     -    0s
     0     0    

In [56]:
for r in rows:
    if (r - 1) % 3 == 0:
        print('+-------+-------+-------+')
    line = ''
    for c in columns:
        if (c - 1) % 3 == 0:
            line += '| '
        line += f'{int(y[r, c].X)} '
        if c == 9:
            line += '|'
    print(line)
print('+-------+-------+-------+')

+-------+-------+-------+
| 8 1 2 | 7 5 3 | 6 4 9 |
| 9 4 3 | 6 8 2 | 1 7 5 |
| 6 7 5 | 4 9 1 | 2 8 3 |
+-------+-------+-------+
| 1 5 4 | 2 3 7 | 8 9 6 |
| 3 6 9 | 8 4 5 | 7 2 1 |
| 2 8 7 | 1 6 9 | 5 3 4 |
+-------+-------+-------+
| 5 2 1 | 9 7 4 | 3 6 8 |
| 4 3 8 | 5 2 6 | 9 1 7 |
| 7 9 6 | 3 1 8 | 4 5 2 |
+-------+-------+-------+


# Sudoku - Binary Formulation

## Data

In [3]:
values = range(1, 10)

In [4]:
rows = range(1, 10)
columns = range(1, 10)
boxes = [
    [(3*i + k, 3*j + l) for k in range(1, 4) for l in range(1, 4)]
    for i in range(3) for j in range(3)
]

## Model

In [14]:
m = Model("Sudoku binary")

In [15]:
y = m.addVars(rows, columns, values, vtype=GRB.BINARY)

In [16]:
m.addConstrs(
    y[r, c, v] == 1 for ((r, c), v) in init_nums.items()
)
m.update()

One value per entry

In [17]:
m.addConstrs(
    sum(y[r, c, v] for v in values) == 1 for r in rows for c in columns
)
m.update()

uniqe value per row

In [21]:
m.addConstrs(
    sum(y[r, c, v] for c in columns) == 1 for r in rows for v in values
)
m.update()

unique value per column

In [24]:
m.addConstrs(
    sum(y[r, c, v] for r in rows) == 1 for c in columns for v in values
)
m.update()

unique value per box

In [29]:
for box in boxes:
    m.addConstrs(
        sum(y[r, c, v] for (r, c) in box) == 1 for v in values
    )
m.update()

In [30]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 427 rows, 729 columns and 3675 nonzeros
Model fingerprint: 0x8c48c1f9
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R345 by 1.000000000

Presolve removed 214 rows and 493 columns
Presolve time: 0.01s
Presolved: 213 rows, 236 columns, 940 nonzeros
Variable types: 0 continuous, 236 integer (236 binary)

Root relaxation: objective 0.000000e+00, 249 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  101          -    0.00000      -     -    0s
H    0     0                    

In [31]:
for r in rows:
    if (r - 1) % 3 == 0:
        print('+-------+-------+-------+')
    line = ''
    for c in columns:
        if (c - 1) % 3 == 0:
            line += '| '
        val = int(sum(y[r, c, v].X * v for v in values))
        line += '{} '.format(val)
        if c == 9:
            line += '|'
    print(line)
print('+-------+-------+-------+')

+-------+-------+-------+
| 8 1 2 | 7 5 3 | 6 4 9 |
| 9 4 3 | 6 8 2 | 1 7 5 |
| 6 7 5 | 4 9 1 | 2 8 3 |
+-------+-------+-------+
| 1 5 4 | 2 3 7 | 8 9 6 |
| 3 6 9 | 8 4 5 | 7 2 1 |
| 2 8 7 | 1 6 9 | 5 3 4 |
+-------+-------+-------+
| 5 2 1 | 9 7 4 | 3 6 8 |
| 4 3 8 | 5 2 6 | 9 1 7 |
| 7 9 6 | 3 1 8 | 4 5 2 |
+-------+-------+-------+
